***GENERATED CODE FOR alitestchurnclass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'}], 'feature': 'Customer ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '0002-ORFBO', 'max': '0730-BGQGF', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer ID'}, {'feature_label': 'Customer ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Married', 'transformation_label': 'String Indexer'}], 'feature': 'Married', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Married'}, {'feature_label': 'Married', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Married')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aguanga', 'max': 'Yreka', 'missing': '0', 'distinct': '350'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Offer', 'transformation_label': 'String Indexer'}], 'feature': 'Offer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'None', 'max': 'Offer E', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Offer'}, {'feature_label': 'Offer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Offer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone Service', 'transformation_label': 'String Indexer'}], 'feature': 'Phone Service', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone Service'}, {'feature_label': 'Phone Service', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone Service')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Multiple Lines', 'transformation_label': 'String Indexer'}], 'feature': 'Multiple Lines', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '452', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '48', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Multiple Lines'}, {'feature_label': 'Multiple Lines', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Multiple Lines')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Internet Service', 'transformation_label': 'String Indexer'}], 'feature': 'Internet Service', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Internet Service'}, {'feature_label': 'Internet Service', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Internet Service')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Internet Type', 'transformation_label': 'String Indexer'}], 'feature': 'Internet Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'Cable', 'max': 'Fiber Optic', 'missing': '92', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Internet Type'}, {'feature_label': 'Internet Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Internet Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Online Security', 'transformation_label': 'String Indexer'}], 'feature': 'Online Security', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Online Security'}, {'feature_label': 'Online Security', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Online Security')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Online Backup', 'transformation_label': 'String Indexer'}], 'feature': 'Online Backup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Online Backup'}, {'feature_label': 'Online Backup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Online Backup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Device Protection Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Device Protection Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Device Protection Plan'}, {'feature_label': 'Device Protection Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Device Protection Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Premium Tech Support', 'transformation_label': 'String Indexer'}], 'feature': 'Premium Tech Support', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Premium Tech Support'}, {'feature_label': 'Premium Tech Support', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Premium Tech Support')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Streaming TV', 'transformation_label': 'String Indexer'}], 'feature': 'Streaming TV', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Streaming TV'}, {'feature_label': 'Streaming TV', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Streaming TV')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Streaming Movies', 'transformation_label': 'String Indexer'}], 'feature': 'Streaming Movies', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Streaming Movies'}, {'feature_label': 'Streaming Movies', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Streaming Movies')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Streaming Music', 'transformation_label': 'String Indexer'}], 'feature': 'Streaming Music', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Streaming Music'}, {'feature_label': 'Streaming Music', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Streaming Music')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Unlimited Data', 'transformation_label': 'String Indexer'}], 'feature': 'Unlimited Data', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '92', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Unlimited Data'}, {'feature_label': 'Unlimited Data', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Unlimited Data')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Contract', 'transformation_label': 'String Indexer'}], 'feature': 'Contract', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Month-to-Month', 'max': 'Two Year', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Contract'}, {'feature_label': 'Contract', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Contract')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Paperless Billing', 'transformation_label': 'String Indexer'}], 'feature': 'Paperless Billing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Paperless Billing'}, {'feature_label': 'Paperless Billing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Paperless Billing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Payment Method', 'transformation_label': 'String Indexer'}], 'feature': 'Payment Method', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bank Withdrawal', 'max': 'Mailed Check', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Payment Method'}, {'feature_label': 'Payment Method', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Payment Method')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Status', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Churned', 'max': 'Stayed', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Status'}, {'feature_label': 'Customer Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Churn Category', 'transformation_label': 'String Indexer'}], 'feature': 'Churn Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '144', 'mean': '', 'stddev': '', 'min': 'Attitude', 'max': 'Price', 'missing': '356', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Churn Category'}, {'feature_label': 'Churn Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Churn Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Churn Reason', 'transformation_label': 'String Indexer'}], 'feature': 'Churn Reason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '144', 'mean': '', 'stddev': '', 'min': 'Attitude of service provider', 'max': 'Service dissatisfaction', 'missing': '356', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Churn Reason'}, {'feature_label': 'Churn Reason', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Churn Reason')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run alitestchurnclassHooks.ipynb
try:
	#sourcePreExecutionHook()

	telecomcustomerchurn = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/telecom_customer_churn.csv', 'filename': 'telecom_customer_churn.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.61.134', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(telecomcustomerchurn)

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run alitestchurnclassHooks.ipynb
try:
	#transformationPreExecutionHook()

	alitestchurnclassautofe = TransformationMain.run(telecomcustomerchurn,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Customer ID", "transformation_label": "String Indexer"}], "feature": "Customer ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0002-ORFBO", "max": "0730-BGQGF", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer ID"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "46.88", "stddev": "17.13", "min": "19", "max": "80", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Married", "transformation_label": "String Indexer"}], "feature": "Married", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Married"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number of Dependents", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.48", "stddev": "1.0", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "Number of Dependents"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aguanga", "max": "Yreka", "missing": "0", "distinct": "350"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "93599.27", "stddev": "1822.54", "min": "90001", "max": "96140", "missing": "0"}, "updatedLabel": "Zip Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "36.28", "stddev": "2.46", "min": "32.555828", "max": "41.813521", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-119.87", "stddev": "2.19", "min": "-124.301372", "max": "-114.758334", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Longitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Number of Referrals", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.0", "stddev": "3.1", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "Number of Referrals"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Tenure in Months", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "33.59", "stddev": "24.53", "min": "1", "max": "72", "missing": "0"}, "updatedLabel": "Tenure in Months"}, {"transformationsData": [{"feature_label": "Offer", "transformation_label": "String Indexer"}], "feature": "Offer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "None", "max": "Offer E", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Offer"}, {"transformationsData": [{"feature_label": "Phone Service", "transformation_label": "String Indexer"}], "feature": "Phone Service", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone Service"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Avg Monthly Long Distance Charges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "452", "mean": "25.58", "stddev": "14.53", "min": "1.09", "max": "49.96", "missing": "48"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Avg Monthly Long Distance..."}, {"transformationsData": [{"feature_label": "Multiple Lines", "transformation_label": "String Indexer"}], "feature": "Multiple Lines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "452", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "48", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Multiple Lines"}, {"transformationsData": [{"feature_label": "Internet Service", "transformation_label": "String Indexer"}], "feature": "Internet Service", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Internet Service"}, {"transformationsData": [{"feature_label": "Internet Type", "transformation_label": "String Indexer"}], "feature": "Internet Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "Cable", "max": "Fiber Optic", "missing": "92", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Internet Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Avg Monthly GB Download", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "408", "mean": "26.07", "stddev": "19.53", "min": "2", "max": "85", "missing": "92"}, "updatedLabel": "Avg Monthly GB Download"}, {"transformationsData": [{"feature_label": "Online Security", "transformation_label": "String Indexer"}], "feature": "Online Security", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Online Security"}, {"transformationsData": [{"feature_label": "Online Backup", "transformation_label": "String Indexer"}], "feature": "Online Backup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Online Backup"}, {"transformationsData": [{"feature_label": "Device Protection Plan", "transformation_label": "String Indexer"}], "feature": "Device Protection Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Device Protection Plan"}, {"transformationsData": [{"feature_label": "Premium Tech Support", "transformation_label": "String Indexer"}], "feature": "Premium Tech Support", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Premium Tech Support"}, {"transformationsData": [{"feature_label": "Streaming TV", "transformation_label": "String Indexer"}], "feature": "Streaming TV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Streaming TV"}, {"transformationsData": [{"feature_label": "Streaming Movies", "transformation_label": "String Indexer"}], "feature": "Streaming Movies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Streaming Movies"}, {"transformationsData": [{"feature_label": "Streaming Music", "transformation_label": "String Indexer"}], "feature": "Streaming Music", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Streaming Music"}, {"transformationsData": [{"feature_label": "Unlimited Data", "transformation_label": "String Indexer"}], "feature": "Unlimited Data", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "92", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Unlimited Data"}, {"transformationsData": [{"feature_label": "Contract", "transformation_label": "String Indexer"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Month-to-Month", "max": "Two Year", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Contract"}, {"transformationsData": [{"feature_label": "Paperless Billing", "transformation_label": "String Indexer"}], "feature": "Paperless Billing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Paperless Billing"}, {"transformationsData": [{"feature_label": "Payment Method", "transformation_label": "String Indexer"}], "feature": "Payment Method", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bank Withdrawal", "max": "Mailed Check", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Payment Method"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Monthly Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "67.01", "stddev": "30.45", "min": "-10.0", "max": "116.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Monthly Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Charges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2436.52", "stddev": "2284.75", "min": "19.7", "max": "8543.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Charges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Refunds", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.76", "stddev": "7.12", "min": "0.0", "max": "49.76", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Refunds"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Extra Data Charges", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.08", "stddev": "25.68", "min": "0", "max": "150", "missing": "0"}, "updatedLabel": "Total Extra Data Charges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Long Distance Charges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "783.55", "stddev": "870.21", "min": "0.0", "max": "3482.64", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Long Distance Charg..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Revenue", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3225.39", "stddev": "2874.21", "min": "25.05", "max": "11979.34", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total Revenue"}, {"transformationsData": [{"feature_label": "Customer Status", "transformation_label": "String Indexer"}], "feature": "Customer Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Churned", "max": "Stayed", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Status"}, {"transformationsData": [{"feature_label": "Churn Category", "transformation_label": "String Indexer"}], "feature": "Churn Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "144", "mean": "", "stddev": "", "min": "Attitude", "max": "Price", "missing": "356", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Churn Category"}, {"transformationsData": [{"feature_label": "Churn Reason", "transformation_label": "String Indexer"}], "feature": "Churn Reason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "144", "mean": "", "stddev": "", "min": "Attitude of service provider", "max": "Service dissatisfaction", "missing": "356", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Churn Reason"}]}))

	#transformationPostExecutionHook(alitestchurnclassautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run alitestchurnclassHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(alitestchurnclassautofe, ["Age", "Number of Dependents", "Zip Code", "Latitude", "Longitude", "Number of Referrals", "Tenure in Months", "Avg Monthly Long Distance Charges", "Avg Monthly GB Download", "Monthly Charge", "Total Charges", "Total Refunds", "Total Extra Data Charges", "Total Long Distance Charges", "Total Revenue", "Customer ID_stringindexer", "Gender_stringindexer", "Married_stringindexer", "City_stringindexer", "Offer_stringindexer", "Phone Service_stringindexer", "Multiple Lines_stringindexer", "Internet Service_stringindexer", "Internet Type_stringindexer", "Online Security_stringindexer", "Online Backup_stringindexer", "Device Protection Plan_stringindexer", "Premium Tech Support_stringindexer", "Streaming TV_stringindexer", "Streaming Movies_stringindexer", "Streaming Music_stringindexer", "Unlimited Data_stringindexer", "Contract_stringindexer", "Paperless Billing_stringindexer", "Payment Method_stringindexer", "Customer Status_stringindexer", "Churn Reason_stringindexer"], "Churn Category_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

